In [31]:
import spacy
from heapq import nlargest

In [2]:
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [3]:
text="""Game of Thrones is an American fantasy drama television series created by David Benioff and D. B. Weiss for HBO. It is an adaptation of A Song of Ice and Fire, a series of fantasy novels by George R. R. Martin, the first of which is A Game of Thrones. The show premiered on HBO in the United States on April 17, 2011, and concluded on May 19, 2019, with 73 episodes broadcast over eight seasons.

Set on the fictional continents of Westeros and Essos, Game of Thrones has a large ensemble cast and follows several story arcs throughout the course of the show. The first major arc concerns the Iron Throne of the Seven Kingdoms of Westeros through a web of political conflicts among the noble families either vying to claim the throne or fighting for independence from whoever sits on it.

 The second major arc focuses on the last descendant of the realm's deposed ruling dynasty, who has been exiled to Essos and is plotting to return and reclaim the throne. The third follows the Night's Watch, a military order defending the realm against threats from beyond the Seven Kingdoms' northern border."""

In [4]:
def summarizer(rawdocs): 
     stopwords=list( STOP_WORDS)
     print(stopwords)

['per', 'becoming', 'used', 'along', 'still', '’m', 'only', "'ve", 'meanwhile', 'hereupon', '‘m', 'then', 'fifteen', 'regarding', 'namely', 'herself', 'over', 'onto', 'whole', 'rather', 'none', 'part', 'she', 'until', 'became', 'again', 'he', 'or', 'keep', 'enough', 'by', 'full', 'has', 'itself', 'seemed', 'is', 'it', 'say', 'yourself', 'via', 'was', 'that', 'me', 'with', 'last', '’s', 'side', 'since', 'about', 'mostly', 'thereupon', 'any', '’d', 'our', 'ours', 'amount', 'hers', 'after', 'mine', 'thereafter', 'whereafter', 'have', 'below', 'anyway', 'indeed', 'everything', 'noone', '‘d', 'two', 'be', 'am', 'either', 'being', 'least', 'no', 'never', 'your', 'while', 'even', 'n‘t', 'the', 'six', 'anywhere', 'but', "'s", 'doing', 'may', 'fifty', 'amongst', 'had', 'thru', 'give', 'many', 'put', 'further', 'several', 'an', 'beyond', 'both', 'name', 'around', 'serious', 'whoever', 'anything', 'seems', 'hereafter', 'well', 'cannot', '‘s', 'too', 'most', 'ourselves', 'next', 'own', 'back', 'af

In [5]:
#smallest Module of Spacy 
nlp=spacy.load('en_core_web_sm')

In [6]:
doc=nlp(text)

In [9]:
tokens=[token.text for token in doc]
print(tokens)

['Game', 'of', 'Thrones', 'is', 'an', 'American', 'fantasy', 'drama', 'television', 'series', 'created', 'by', 'David', 'Benioff', 'and', 'D.', 'B.', 'Weiss', 'for', 'HBO', '.', 'It', 'is', 'an', 'adaptation', 'of', 'A', 'Song', 'of', 'Ice', 'and', 'Fire', ',', 'a', 'series', 'of', 'fantasy', 'novels', 'by', 'George', 'R.', 'R.', 'Martin', ',', 'the', 'first', 'of', 'which', 'is', 'A', 'Game', 'of', 'Thrones', '.', 'The', 'show', 'premiered', 'on', 'HBO', 'in', 'the', 'United', 'States', 'on', 'April', '17', ',', '2011', ',', 'and', 'concluded', 'on', 'May', '19', ',', '2019', ',', 'with', '73', 'episodes', 'broadcast', 'over', 'eight', 'seasons', '.', '\n\n', 'Set', 'on', 'the', 'fictional', 'continents', 'of', 'Westeros', 'and', 'Essos', ',', 'Game', 'of', 'Thrones', 'has', 'a', 'large', 'ensemble', 'cast', 'and', 'follows', 'several', 'story', 'arcs', 'throughout', 'the', 'course', 'of', 'the', 'show', '.', 'The', 'first', 'major', 'arc', 'concerns', 'the', 'Iron', 'Throne', 'of', '

In [10]:
# it picks up each word in from doc and convert it into lower and check if it is in stopwords and punctuations.
word_freq={}
# each word will be added into dictionary and assign a valuw to each key 
for word in doc:
     if word.text.lower() not in stopwords and word.text.lower() not in punctuation:
          # if not then convert word into text and check if word is in word_freq's dictionary if not then assign that word 1 and if repeat again then increase its value .
          if word.text not in word_freq.keys():
               word_freq[word.text]=1
          else:
               word_freq[word.text]+=1

In [11]:
print(word_freq)

{'Game': 3, 'Thrones': 3, 'American': 1, 'fantasy': 2, 'drama': 1, 'television': 1, 'series': 2, 'created': 1, 'David': 1, 'Benioff': 1, 'D.': 1, 'B.': 1, 'Weiss': 1, 'HBO': 2, 'adaptation': 1, 'Song': 1, 'Ice': 1, 'Fire': 1, 'novels': 1, 'George': 1, 'R.': 2, 'Martin': 1, 'premiered': 1, 'United': 1, 'States': 1, 'April': 1, '17': 1, '2011': 1, 'concluded': 1, '19': 1, '2019': 1, '73': 1, 'episodes': 1, 'broadcast': 1, 'seasons': 1, '\n\n': 1, 'Set': 1, 'fictional': 1, 'continents': 1, 'Westeros': 2, 'Essos': 2, 'large': 1, 'ensemble': 1, 'cast': 1, 'follows': 2, 'story': 1, 'arcs': 1, 'course': 1, 'major': 2, 'arc': 2, 'concerns': 1, 'Iron': 1, 'Throne': 1, 'Seven': 2, 'Kingdoms': 2, 'web': 1, 'political': 1, 'conflicts': 1, 'noble': 1, 'families': 1, 'vying': 1, 'claim': 1, 'throne': 2, 'fighting': 1, 'independence': 1, 'sits': 1, '\n\n ': 1, 'second': 1, 'focuses': 1, 'descendant': 1, 'realm': 2, 'deposed': 1, 'ruling': 1, 'dynasty': 1, 'exiled': 1, 'plotting': 1, 'return': 1, 'rec

In [14]:
max_freq=max(word_freq.values())
print(max_freq)

3


dict_keys(['Game', 'Thrones', 'American', 'fantasy', 'drama', 'television', 'series', 'created', 'David', 'Benioff', 'D.', 'B.', 'Weiss', 'HBO', 'adaptation', 'Song', 'Ice', 'Fire', 'novels', 'George', 'R.', 'Martin', 'premiered', 'United', 'States', 'April', '17', '2011', 'concluded', '19', '2019', '73', 'episodes', 'broadcast', 'seasons', '\n\n', 'Set', 'fictional', 'continents', 'Westeros', 'Essos', 'large', 'ensemble', 'cast', 'follows', 'story', 'arcs', 'course', 'major', 'arc', 'concerns', 'Iron', 'Throne', 'Seven', 'Kingdoms', 'web', 'political', 'conflicts', 'noble', 'families', 'vying', 'claim', 'throne', 'fighting', 'independence', 'sits', '\n\n ', 'second', 'focuses', 'descendant', 'realm', 'deposed', 'ruling', 'dynasty', 'exiled', 'plotting', 'return', 'reclaim', 'Night', 'Watch', 'military', 'order', 'defending', 'threats', 'northern', 'border'])

In [16]:
#normalized Frequency = Frequency of each word / Max frequency 
for word in word_freq.keys():
     word_freq[word]=word_freq[word]/max_freq


In [17]:
print(word_freq)

{'Game': 1.0, 'Thrones': 1.0, 'American': 0.3333333333333333, 'fantasy': 0.6666666666666666, 'drama': 0.3333333333333333, 'television': 0.3333333333333333, 'series': 0.6666666666666666, 'created': 0.3333333333333333, 'David': 0.3333333333333333, 'Benioff': 0.3333333333333333, 'D.': 0.3333333333333333, 'B.': 0.3333333333333333, 'Weiss': 0.3333333333333333, 'HBO': 0.6666666666666666, 'adaptation': 0.3333333333333333, 'Song': 0.3333333333333333, 'Ice': 0.3333333333333333, 'Fire': 0.3333333333333333, 'novels': 0.3333333333333333, 'George': 0.3333333333333333, 'R.': 0.6666666666666666, 'Martin': 0.3333333333333333, 'premiered': 0.3333333333333333, 'United': 0.3333333333333333, 'States': 0.3333333333333333, 'April': 0.3333333333333333, '17': 0.3333333333333333, '2011': 0.3333333333333333, 'concluded': 0.3333333333333333, '19': 0.3333333333333333, '2019': 0.3333333333333333, '73': 0.3333333333333333, 'episodes': 0.3333333333333333, 'broadcast': 0.3333333333333333, 'seasons': 0.333333333333333

In [20]:
# sentence tokenize  
sent_tokens=[sent for sent in doc.sents]
print(sent_tokens)

[Game of Thrones is an American fantasy drama television series created by David Benioff and D. B. Weiss for HBO., It is an adaptation of A Song of Ice and Fire, a series of fantasy novels by George R. R. Martin, the first of which is A Game of Thrones., The show premiered on HBO in the United States on April 17, 2011, and concluded on May 19, 2019, with 73 episodes broadcast over eight seasons.

, Set on the fictional continents of Westeros and Essos, Game of Thrones has a large ensemble cast and follows several story arcs throughout the course of the show., The first major arc concerns the Iron Throne of the Seven Kingdoms of Westeros through a web of political conflicts among the noble families either vying to claim the throne or fighting for independence from whoever sits on it.

 , The second major arc focuses on the last descendant of the realm's deposed ruling dynasty, who has been exiled to Essos and is plotting to return and reclaim the throne., The third follows the Night's W

In [27]:
# make dictionary for sentence in similar manner
sent_score={}
for sent in sent_tokens:  # picking each sentence from token 
  for word in sent:     # picking each ford from tokenized sentence 
    if word.text in word_freq.keys():  # check each word in sent if in word_freq dictionary  if word exist then 
      if sent not in sent_score.keys():  # check if sentence exist in sent_score dictionary if not exist then add word_freq word in sent_score
            sent_score[sent]=word_freq[word.text]  # assign value 
      else:
            sent_score[sent] +=word_freq[word.text]  # add the normalize frequency 

            # it check each word in sentence if word come again then it increases  its value and then go for next word and add the prev word in sentence and it add the overall frequency
            #  
            #In this way it calculate total frequency of each sentence


In [28]:
sent_score # total frequnecy of each sentence 

{Game of Thrones is an American fantasy drama television series created by David Benioff and D. B. Weiss for HBO.: 6.999999999999999,
 It is an adaptation of A Song of Ice and Fire, a series of fantasy novels by George R. R. Martin, the first of which is A Game of Thrones.: 7.0,
 The show premiered on HBO in the United States on April 17, 2011, and concluded on May 19, 2019, with 73 episodes broadcast over eight seasons.
 : 5.333333333333332,
 Set on the fictional continents of Westeros and Essos, Game of Thrones has a large ensemble cast and follows several story arcs throughout the course of the show.: 6.999999999999998,
 The first major arc concerns the Iron Throne of the Seven Kingdoms of Westeros through a web of political conflicts among the noble families either vying to claim the throne or fighting for independence from whoever sits on it.
 
  : 8.666666666666664,
 The second major arc focuses on the last descendant of the realm's deposed ruling dynasty, who has been exiled to 

In [29]:
select_len=int(len(sent_tokens)*0.3)  # 30% length of sent token 
print(select_len)

2


In [32]:
# it brought select_len sentences from sent_scores and get the sent of highest freq from sent score
summary=nlargest(select_len,sent_score,key=sent_score.get)

In [33]:
# sentence with highest summary will be selected 
summary


[The first major arc concerns the Iron Throne of the Seven Kingdoms of Westeros through a web of political conflicts among the noble families either vying to claim the throne or fighting for independence from whoever sits on it.
 
  ,
 It is an adaptation of A Song of Ice and Fire, a series of fantasy novels by George R. R. Martin, the first of which is A Game of Thrones.]

In [34]:
final_summary=[word.text for word in summary] # make list using word.text of each word and join each word by space 
summary=' '.join(final_summary)

In [36]:
print("Original Text is : " )
print(text)
print("_________________________________________________________________________________________________")
print("summarized Text is :")
print(summary)
print("_________________________________________________________________________________________________")


Original Text is : 
Game of Thrones is an American fantasy drama television series created by David Benioff and D. B. Weiss for HBO. It is an adaptation of A Song of Ice and Fire, a series of fantasy novels by George R. R. Martin, the first of which is A Game of Thrones. The show premiered on HBO in the United States on April 17, 2011, and concluded on May 19, 2019, with 73 episodes broadcast over eight seasons.

Set on the fictional continents of Westeros and Essos, Game of Thrones has a large ensemble cast and follows several story arcs throughout the course of the show. The first major arc concerns the Iron Throne of the Seven Kingdoms of Westeros through a web of political conflicts among the noble families either vying to claim the throne or fighting for independence from whoever sits on it.

 The second major arc focuses on the last descendant of the realm's deposed ruling dynasty, who has been exiled to Essos and is plotting to return and reclaim the throne. The third follows th

In [37]:
print("Length Of Original Text :", len(text.split(' ')) )
print("Length Of Summarized Text :", len(summary.split(' ')) )

Length Of Original Text : 193
Length Of Summarized Text : 70
